<a href="https://colab.research.google.com/github/anushka-code/Code-Smell-Classification/blob/main/Code_Smell_Classification_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Structural Branch of the NN-Architecture : 1d-CNN for Structural Object Oriented Metric Feature Extraction

##Code Smells Targetted: 


1.   Long Parameters List
2.   Switch Statements







###Install Libraries

In [22]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Convolution1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.utils import np_utils

###Mounting Google Drive

In [1]:
!pip install -U -q PyDrive

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
    
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

###Dataset Loader

In [29]:
def DataLoader(link, name_of_file):
  id = link.split("/")[-2]
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile(name_of_file)
  dataframe = pd.read_csv(name_of_file)
  return dataframe

link1 = 'https://drive.google.com/file/d/1EfbAqgr7i9h4yFwEoU3igG34Gt48l6WT/view?usp=sharing'
link2 = 'https://drive.google.com/file/d/1Ya1OMWsz1yyXAaZheIck-roX0M9UWiqg/view?usp=sharing'
name1 = 'long_parameters_list_structural.csv'
name2 = 'switch_statements_structural.csv'

Long Parameters List Dataset

In [30]:
df_lp = DataLoader(link1, name1)
df_lp.head(5)

,NOP_method,CC_method,ATFD_method,FDP_method,CM_method,MAXNESTING_method,LOC_method,CYCLO_method,NMCS_method,NOLV_method,MaMCL_method,NOAV_method,LAA_method,FANOUT_method,CFNAMM_method,ATLD_method,CLNAMM_method,CINT_method,MeMCL_method,CDISP_method,NOII_type,NOAM_type,NOCS_type,NOM_type,NMO_type,ATFD_type,FANOUT_type,NOMNAMM_type,NOA_type,NIM_type,DIT_type,LOC_type,LOCNAMM_type,CFNAMM_type,TCC_type,NOPA_type,CBO_type,RFC_type,NOC_type,WMC_type,LCOM5_type,WOC_type,WMCNAMM_type,AMW_type,AMWNAMM_type,NOCS_package,NOMNAMM_package,NOI_package,LOC_package,NOM_package,NOPK_project,NOCS_project,NOI_project,NOM_project,NOMNAMM_project,LOC_project,is_long_parameters_list
0,0,0,0,0,0,2,8,2,0,0,0,6,1,0,2,3,0,0,0,0.0,0,7,2,16,?,2,3,9,12,?,4,221,172,406,0.177778,1,9,34,1,29,0.961111,?,18,1.812500,2.000000,6,9,0,229,16,43,849,78,5788,4778,65687,False
1,0,0,0,0,0,1,3,1,0,0,0,1,1,0,0,1,0,0,0,0.0,1,0,0,3,1,0,0,3,1,0,1,16,16,0,1.000000,0,0,3,0,3,0.500000,1,3,1.000000,1.000000,38,198,0,4223,240,43,849,78,5788,4778,65687,False
2,1,0,0,0,0,1,2,1,0,1,0,1,0,1,3,0,0,1,0,1.0,1,29,18,123,17,50,138,94,108,77,2,1957,1669,51801,0.012613,87,145,571,0,273,0.992107,0.404145078,234,2.219512,2.489362,38,198,0,4223,240,43,849,78,5788,4778,65687,False
3,0,5,0,0,9,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0.0,1,21,0,92,?,0,0,71,5,?,?,288,206,0,0.000000,5,15,92,0,92,1.010989,0,71,1.000000,1.000000,34,523,11,6037,728,43,849,78,5788,4778,65687,False
4,1,0,1,1,0,1,3,1,0,1,0,2,0,1,0,0,0,1,0,1.0,1,0,0,3,0,1,1,3,0,0,1,12,12,0,0.000000,0,1,4,0,3,0.000000,1,3,1.000000,1.000000,34,523,11,6037,728,43,849,78,5788,4778,65687,False


Switch Statements Dataset

In [31]:
df_ss = DataLoader(link1, name1)
df_ss.head(5)

,NOP_method,CC_method,ATFD_method,FDP_method,CM_method,MAXNESTING_method,LOC_method,CYCLO_method,NMCS_method,NOLV_method,MaMCL_method,NOAV_method,LAA_method,FANOUT_method,CFNAMM_method,ATLD_method,CLNAMM_method,CINT_method,MeMCL_method,CDISP_method,NOII_type,NOAM_type,NOCS_type,NOM_type,NMO_type,ATFD_type,FANOUT_type,NOMNAMM_type,NOA_type,NIM_type,DIT_type,LOC_type,LOCNAMM_type,CFNAMM_type,TCC_type,NOPA_type,CBO_type,RFC_type,NOC_type,WMC_type,LCOM5_type,WOC_type,WMCNAMM_type,AMW_type,AMWNAMM_type,NOCS_package,NOMNAMM_package,NOI_package,LOC_package,NOM_package,NOPK_project,NOCS_project,NOI_project,NOM_project,NOMNAMM_project,LOC_project,is_long_parameters_list
0,0,0,0,0,0,2,8,2,0,0,0,6,1,0,2,3,0,0,0,0.0,0,7,2,16,?,2,3,9,12,?,4,221,172,406,0.177778,1,9,34,1,29,0.961111,?,18,1.812500,2.000000,6,9,0,229,16,43,849,78,5788,4778,65687,False
1,0,0,0,0,0,1,3,1,0,0,0,1,1,0,0,1,0,0,0,0.0,1,0,0,3,1,0,0,3,1,0,1,16,16,0,1.000000,0,0,3,0,3,0.500000,1,3,1.000000,1.000000,38,198,0,4223,240,43,849,78,5788,4778,65687,False
2,1,0,0,0,0,1,2,1,0,1,0,1,0,1,3,0,0,1,0,1.0,1,29,18,123,17,50,138,94,108,77,2,1957,1669,51801,0.012613,87,145,571,0,273,0.992107,0.404145078,234,2.219512,2.489362,38,198,0,4223,240,43,849,78,5788,4778,65687,False
3,0,5,0,0,9,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0.0,1,21,0,92,?,0,0,71,5,?,?,288,206,0,0.000000,5,15,92,0,92,1.010989,0,71,1.000000,1.000000,34,523,11,6037,728,43,849,78,5788,4778,65687,False
4,1,0,1,1,0,1,3,1,0,1,0,2,0,1,0,0,0,1,0,1.0,1,0,0,3,0,1,1,3,0,0,1,12,12,0,0.000000,0,1,4,0,3,0.000000,1,3,1.000000,1.000000,34,523,11,6037,728,43,849,78,5788,4778,65687,False


### Data Pre-Processing of the Code Smell Datasets

In [6]:
df_lp.rename(columns = {'is_long_parameters_list':'is_code_smell'}, inplace = True)
df_lp["is_code_smell"] = df_lp["is_code_smell"].astype(int)
Y_lp = df_lp.iloc[:,-1:]
X_lp = df_lp.iloc[:,:56]
X_lp = X_lp.replace(to_replace =["?"], value = np.nan)
X_lp = X_lp.astype(float)

In [7]:
X_lp.head(5)

,NOP_method,CC_method,ATFD_method,FDP_method,CM_method,MAXNESTING_method,LOC_method,CYCLO_method,NMCS_method,NOLV_method,MaMCL_method,NOAV_method,LAA_method,FANOUT_method,CFNAMM_method,ATLD_method,CLNAMM_method,CINT_method,MeMCL_method,CDISP_method,NOII_type,NOAM_type,NOCS_type,NOM_type,NMO_type,ATFD_type,FANOUT_type,NOMNAMM_type,NOA_type,NIM_type,DIT_type,LOC_type,LOCNAMM_type,CFNAMM_type,TCC_type,NOPA_type,CBO_type,RFC_type,NOC_type,WMC_type,LCOM5_type,WOC_type,WMCNAMM_type,AMW_type,AMWNAMM_type,NOCS_package,NOMNAMM_package,NOI_package,LOC_package,NOM_package,NOPK_project,NOCS_project,NOI_project,NOM_project,NOMNAMM_project,LOC_project
0,0.0,0.0,0.0,0.0,0.0,2.0,8.0,2.0,0.0,0.0,0.0,6.0,1.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,7.0,2.0,16.0,NaN,2.0,3.0,9.0,12.0,NaN,4.0,221.0,172.0,406.0,0.177778,1.0,9.0,34.0,1.0,29.0,0.961111,NaN,18.0,1.812500,2.000000,6.0,9.0,0.0,229.0,16.0,43.0,849.0,78.0,5788.0,4778.0,65687.0
1,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,1.0,0.0,1.0,16.0,16.0,0.0,1.000000,0.0,0.0,3.0,0.0,3.0,0.500000,1.000000,3.0,1.000000,1.000000,38.0,198.0,0.0,4223.0,240.0,43.0,849.0,78.0,5788.0,4778.0,65687.0
2,1.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,1.0,0.0,1.0,1.0,29.0,18.0,123.0,17.0,50.0,138.0,94.0,108.0,77.0,2.0,1957.0,1669.0,51801.0,0.012613,87.0,145.0,571.0,0.0,273.0,0.992107,0.404145,234.0,2.219512,2.489362,38.0,198.0,0.0,4223.0,240.0,43.0,849.0,78.0,5788.0,4778.0,65687.0
3,0.0,5.0,0.0,0.0,9.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,21.0,0.0,92.0,NaN,0.0,0.0,71.0,5.0,NaN,NaN,288.0,206.0,0.0,0.000000,5.0,15.0,92.0,0.0,92.0,1.010989,0.000000,71.0,1.000000,1.000000,34.0,523.0,11.0,6037.0,728.0,43.0,849.0,78.0,5788.0,4778.0,65687.0
4,1.0,0.0,1.0,1.0,0.0,1.0,3.0,1.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,3.0,0.0,1.0,1.0,3.0,0.0,0.0,1.0,12.0,12.0,0.0,0.000000,0.0,1.0,4.0,0.0,3.0,0.000000,1.000000,3.0,1.000000,1.000000,34.0,523.0,11.0,6037.0,728.0,43.0,849.0,78.0,5788.0,4778.0,65687.0


In [8]:
Y_lp.head(5)

,is_code_smell
0,0
1,0
2,0
3,0
4,0


In [9]:
X_lp.shape

(420, 56)

In [10]:
Y_lp.shape

(420, 1)

In [11]:
df_ss.rename(columns = {'is_switch_statements':'is_code_smell'}, inplace = True)
df_ss["is_code_smell"] = df_ss["is_code_smell"].astype(int)
Y_ss = df_ss.iloc[:,-1:]
X_ss = df_ss.iloc[:,:56]
X_ss = X_ss.replace(to_replace =["?"], value = np.nan)
X_ss = X_ss.astype(float)

In [12]:
X_ss.head(5)

,NOP_method,CC_method,ATFD_method,FDP_method,CM_method,MAXNESTING_method,LOC_method,CYCLO_method,NMCS_method,NOLV_method,MaMCL_method,NOAV_method,LAA_method,FANOUT_method,CFNAMM_method,ATLD_method,CLNAMM_method,CINT_method,MeMCL_method,CDISP_method,NOII_type,NOAM_type,NOCS_type,NOM_type,NMO_type,ATFD_type,FANOUT_type,NOMNAMM_type,NOA_type,NIM_type,DIT_type,LOC_type,LOCNAMM_type,CFNAMM_type,TCC_type,NOPA_type,CBO_type,RFC_type,NOC_type,WMC_type,LCOM5_type,WOC_type,WMCNAMM_type,AMW_type,AMWNAMM_type,NOCS_package,NOMNAMM_package,NOI_package,LOC_package,NOM_package,NOPK_project,NOCS_project,NOI_project,NOM_project,NOMNAMM_project,LOC_project
0,1.0,2.0,0.0,0.0,2.0,1.0,5.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,4.0,0.0,0.0,2.0,0.0,0.5,0.0,2.0,0.0,7.0,NaN,1.0,3.0,5.0,2.0,NaN,3.0,53.0,39.0,55.0,0.100000,0.0,4.0,13.0,2.0,8.0,0.916667,NaN,6.0,1.142857,1.200000,7.0,47.0,3.0,439.0,53.0,43.0,849.0,78.0,5788.0,4778.0,65687.0
1,0.0,0.0,0.0,0.0,0.0,2.0,6.0,3.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,30.0,12.0,4.0,10.0,22.0,8.0,111.0,3.0,377.0,310.0,1214.0,0.057143,0.0,32.0,208.0,0.0,61.0,0.943966,0.681818,51.0,2.033333,2.318182,38.0,198.0,0.0,4223.0,240.0,43.0,849.0,78.0,5788.0,4778.0,65687.0
2,2.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,2.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,3.0,3.0,4.0,1.000000,0.0,3.0,2.0,NaN,1.0,0.000000,NaN,1.0,1.000000,1.000000,38.0,198.0,0.0,4223.0,240.0,43.0,849.0,78.0,5788.0,4778.0,65687.0
3,0.0,0.0,0.0,0.0,0.0,2.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,5.0,5.0,4.0,1.000000,0.0,1.0,2.0,NaN,1.0,0.000000,NaN,1.0,1.000000,1.000000,18.0,102.0,0.0,1212.0,120.0,43.0,849.0,78.0,5788.0,4778.0,65687.0
4,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,16.0,0.0,7.0,10.0,14.0,6.0,8.0,3.0,197.0,179.0,640.0,0.305556,0.0,10.0,41.0,0.0,31.0,0.855556,1.000000,28.0,1.937500,2.000000,4.0,27.0,0.0,516.0,29.0,43.0,849.0,78.0,5788.0,4778.0,65687.0


In [13]:
Y_ss.head(5)

,is_code_smell
0,0
1,0
2,0
3,0
4,0


In [14]:
X_ss.shape

(420, 56)

In [15]:
Y_ss.shape

(420, 1)

### Concatenating the Datasets

In [16]:
code_smells = [X_lp, X_ss]
X_train = pd.concat(code_smells)
type(X_train)

pandas.core.frame.DataFrame

In [17]:
X_train.shape
X_train = pd.DataFrame(X_train)

In [18]:
labels = [Y_lp, Y_ss]
Y_train = pd.concat(labels)

In [19]:
Y_train.shape

(840, 1)

In [20]:
Y_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 840 entries, 0 to 419
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   is_code_smell  840 non-null    int64
dtypes: int64(1)
memory usage: 13.1 KB


In [21]:
# Replacing NaN values with means of the columns

X_train[2] = X_train[2].fillna(X_train[2].mean())
X_train[3] = X_train[3].fillna(X_train[3].mean())
X_train[11] = X_train[11].fillna(X_train[11].mean())
X_train[12] = X_train[12].fillna(X_train[12].mean())
X_train[15] = X_train[15].fillna(X_train[15].mean())
X_train[24] = X_train[24].fillna(X_train[24].mean())
X_train[29] = X_train[29].fillna(X_train[29].mean())
X_train[30] = X_train[30].fillna(X_train[30].mean())
X_train[41] = X_train[41].fillna(X_train[41].mean())
X_train[38] = X_train[38].fillna(X_train[38].mean())

KeyError: ignored

###Data Normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# compute required values
scaler = MinMaxScaler()
model = scaler.fit(X_train)
X_train = model.transform(X_train)
type(X_train)

In [ ]:
type(Y_train)

In [ ]:
np.any(np.isnan(X_train))

###Smote's Algorithm for OverSampling

In [ ]:
type(X_train)

In [ ]:
type(Y_train)

In [ ]:
print("Before OverSampling, counts of label '1': {}".format(sum(Y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(Y_train == 0)))
  
# import SMOTE module from imblearn library
# pip install imblearn (if you don't have imblearn in your system)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 0)
oversample = SMOTE()
X_train_res, y_train_res = oversample.fit_resample(X_train, Y_train.ravel())
  
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))
  
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0)))

In [ ]:
X_train_res.shape

In [ ]:
y_train_res.shape

### 1-D CNN Implementation

In [ ]:
X_train_res = X_train_res.to_numpy

In [ ]:
X_train_final = X_train_res.reshape((1146,56,1))

In [ ]:
model = Sequential()
model.add(Convolution1D(filters=128, kernel_size=3, activation='tanh', input_shape=(56,1)))
model.add(Convolution1D(filters=128, kernel_size=3, activation='tanh'))
model.add(Convolution1D(filters=128, kernel_size=5, activation='tanh'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#check pretrained models for feature extraction of CNN 2d data 
#which is good for numerical data
#modelfit xd boost (better version)
#imbalanced data - synthetic data addition 
#random forest

In [ ]:
hist = model.fit(X_train_final,y_train_res,epochs=30,shuffle=True,batch_size=50,validation_split=0.20) 